# Terminology Tagging for Choi et al. (2022)'s Soft Constraint Method

We've obtained our noun alignments for 15% of our training set, and we will now determine the intersection of our filtered glossary and these nouns. Unfortunately, this will take quite a while (about 11 hours).

In [2]:
from datasets import load_dataset, Dataset

#Converts data in src [TAB] tgt [NEWLINE] format to a format suitable for model training
def convertToDictFormat(data):
    source = []
    target = []
    for example in data:
        example = example.strip()
        sentences = example.split("\t")
        source.append(sentences[0])
        target.append(sentences[1])
    ready = Dataset.from_dict({"en":source, "fr":target})
    return ready

In [4]:
#Load in clean glossary and convert to Dataset object
glossary_terms = load_dataset("ethansimrm/choi_filtered_cleaned_glossary", split = "train")
terms_ready = convertToDictFormat(glossary_terms['text'])

Found cached dataset text (C:/Users/ethan/.cache/huggingface/datasets/ethansimrm___text/ethansimrm--choi_filtered_cleaned_glossary-55c8ab5554eb133f/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [19]:
#Read in noun alignments
import pandas as pd
noun_alignments_df = pd.read_csv("noun-alignments.txt", sep = "\t", header = None, names = ["sent_ID", "en", "fr"])

In [29]:
#Take intersection by query
def check_term_pair_in_glossary(en_term, fr_term):
    for gloss_term in terms_ready:
        #Only look for exact matches due to casing differences (glossary accounts for this)
        if ((en_term == gloss_term["en"]) and (fr_term == gloss_term["fr"])):
            return True
    return False

In [ ]:
noun_alignments_df['in_glossary'] = noun_alignments_df.apply(lambda row: check_term_pair_in_glossary(row['en'], row['fr']), axis=1)

In [47]:
#We will filter this later - for now, just check whether a term is present in our glossary, and output the result.
noun_alignments_df.to_csv('found_nouns.txt', sep="\t", header = False, index = False)